In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# 📌 라이브러리

import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import gc

In [8]:
# 📌 메모리 최적화 함수
def optimize_types(df):
    for col in df.select_dtypes(include='float64').columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include='int64').columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

In [9]:
# 📌 1. 테스트 데이터 불러오기
test = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/98제거/test_final_98.csv')

In [10]:
# ID 저장 후 불필요 컬럼 제거
test_id = test['ID']

if 'Unnamed: 0' in test.columns:
    test = test.drop(columns=['Unnamed: 0'])

if 'ID' in test.columns:
    test = test.drop(columns=['ID'])

In [11]:
# 📌 2. 메모리 최적화 및 타입 변환
test = optimize_types(test)

# object → category 변환
for col in test.select_dtypes(include='object').columns:
    test[col] = test[col].astype('category')

In [16]:
with open('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/98제거/X_columns.pkl', 'rb') as f:
    X_columns = pickle.load(f)

In [18]:
test = test[X_columns]

In [20]:
with open('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/98제거/final_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [21]:
preds_proba = model.predict_proba(test)
preds_class = np.argmax(preds_proba, axis=1)

In [23]:
# 숫자 라벨을 문자 라벨로 매핑
label_map = np.array(['A', 'B', 'C', 'D', 'E'])  # le.classes_ 순서

preds_labels = label_map[preds_class]

In [24]:
submission = pd.DataFrame({
    'ID': test_id,
    'Segment': preds_labels
})

submission.to_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/submission.csv', index=False, encoding='utf-8-sig')

print("✅ 제출 파일 저장 완료")

✅ 제출 파일 저장 완료


In [30]:
# 열어서 확인
submission = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/submission.csv')
submission

,ID,Segment
0,TEST_00000,D
1,TEST_00001,E
2,TEST_00002,D
3,TEST_00003,E
4,TEST_00004,E
...,...,...
599995,TEST_99995,E
599996,TEST_99996,E
599997,TEST_99997,E
599998,TEST_99998,C


In [27]:
print("컬럼 목록:", submission.columns.tolist())
print("총 샘플 수:", submission.shape[0])

컬럼 목록: ['ID', 'Segment']
총 샘플 수: 600000


In [31]:
# 마지막 100,000개만 추출
submission_tail = submission.tail(100000)

# 제출 파일 저장
submission_tail.to_csv(
    '/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/submission_tail.csv',
    index=False,
    encoding='utf-8-sig'
)

print("✅ 마지막 100,000개만 제출 파일 저장 완료")

✅ 마지막 100,000개만 제출 파일 저장 완료


In [33]:
print(submission_tail['Segment'].value_counts())
print(submission_tail.shape)

Segment
E    73708
D    18791
C     7450
A       49
B        2
Name: count, dtype: int64
(100000, 2)
